### OAuth2
- 인증을 위한 개방형 표준 프로토콜  
- OAuth2의 주요 목표는 사용자의 아이디와 비밀번호를 공유하지 않고도, 제 3자 애플리케이션에 사용자를 대신해 특정 작업을 수행하도록 허용하는 것  
- Client Credentials Grant 방식: 클라이언트가 자신의 인증 정보(id/pw)를 보내서 액세스 Token을 발급받는 방식.

In [1]:
# 네이버커머스 Auth 정책
# 커머스API는 인증 토큰 발급 요청 시 직접 클라이언트 시크릿 값을 전달하지 않고 전자서명(signature)을 생성 후 전달
# 동일 리소스(type과 account_id에 따라)에는 하나의 인증 토큰의 발급되며 인증 토큰의 유효 시간은 3시간(10,800초)
# 발급된 인증 토큰이 존재하는 경우 남은 유효 시간이 30분 이상이면 기존 인증 토큰이 반환되고 30분 미만이면 새로운 인증 토큰이 발급됩니다
# 새로운 인증 토큰이 발급된 경우에도 유효 시간이 만료되기 전까지는 기존 인증 토큰을 사용할 수 있습니다.

In [1]:
import bcrypt
import pybase64
import requests
import json
import http.client
import pandas as pd
import openpyxl
import xlsxwriter
import time
import datetime
import os
import base64
from itertools import product
import Lib.Common
from Lib.UploadTool import * 
from Lib.SearchTool import *
from Lib.RenewTool import *

In [2]:
access_token

'5Lu2A5oaTTJmm5g0qGvk5E'

# 내 채널 상품 조회

In [3]:
# 스마트스토어센터에서는 "채널상품번호"만을 제공함. -> 채널 상품 번호로 조회해야함 
search_product(9771082183)

'{"originProduct":{"statusType":"SALE","saleType":"NEW","leafCategoryId":"50000836","name":"[Olekei] 사루엘 남성 바지 와이드 팬츠 9부 무지 조정끈 인기 쾌적 넉넉한 통기성 큰 사이즈 캐주얼 하카마 팬츠 편안함 사계절 봄 여름 가을 겨울","detailContent":"<div class=\\"se-viewer se-theme-default\\" lang=\\"ko-KR\\">\\n    <!-- SE_DOC_HEADER_START -->\\n    <!--@CONTENTS_HEADER-->\\n    <!-- SE_DOC_HEADER_END -->\\n    <div class=\\"se-main-container\\">\\n                <div class=\\"se-component se-text se-l-default\\" id=\\"SE-d8431b9e-3e1b-481d-b281-a7a349c2f597\\">\\n                    <div class=\\"se-component-content\\">\\n                        <div class=\\"se-section se-section-text se-l-default\\">\\n                            <div class=\\"se-module se-module-text\\">\\n                                    <!-- SE-TEXT { --><p class=\\"se-text-paragraph se-text-paragraph-align- \\" style=\\"\\" id=\\"SE-25931c06-b52a-435b-afde-55de381d8d9f\\"><span style=\\"color:#000000;background-color:#fdfdfd;\\" class=\\"se-fs-fs24 se-ff-syst

# 카테고리 최신화 및 조회

In [4]:
# 카테고리 리스트 CSV 최신화
# Update_Category_List()

In [5]:
Update_Category_List()

In [ ]:
[x['name'] for x in get_Major_Categories()]
[x['id'] for x in get_Major_Categories()]

In [3]:
get_Major_Categories()

[{'wholeCategoryName': '패션의류',
  'id': '50000000',
  'name': '패션의류',
  'last': False},
 {'wholeCategoryName': '패션잡화',
  'id': '50000001',
  'name': '패션잡화',
  'last': False},
 {'wholeCategoryName': '화장품/미용',
  'id': '50000002',
  'name': '화장품/미용',
  'last': False},
 {'wholeCategoryName': '디지털/가전',
  'id': '50000003',
  'name': '디지털/가전',
  'last': False},
 {'wholeCategoryName': '가구/인테리어',
  'id': '50000004',
  'name': '가구/인테리어',
  'last': False},
 {'wholeCategoryName': '출산/육아',
  'id': '50000005',
  'name': '출산/육아',
  'last': False},
 {'wholeCategoryName': '식품', 'id': '50000006', 'name': '식품', 'last': False},
 {'wholeCategoryName': '스포츠/레저',
  'id': '50000007',
  'name': '스포츠/레저',
  'last': False},
 {'wholeCategoryName': '생활/건강',
  'id': '50000008',
  'name': '생활/건강',
  'last': False},
 {'wholeCategoryName': '여가/생활편의',
  'id': '50000009',
  'name': '여가/생활편의',
  'last': False},
 {'wholeCategoryName': '도서', 'id': '50005542', 'name': '도서', 'last': False}]

In [31]:

url = Base_url + f"/v1/categories/{50000000}/sub-categories"
headers = { 'Authorization': f"Bearer {access_token+'s'}" }
res = requests.get(url, headers=headers,)
ChildCategories = json.loads(res.text)




In [35]:
ChildCategories['message']

'요청을 보낼 권한이 없습니다.'

In [15]:
type(get_Child_Categories("50000000")) == type([])

True

In [8]:
get_Child_Categories("50000170")

[{'wholeCategoryName': '패션의류>남성언더웨어/잠옷>팬티',
  'id': '50000845',
  'name': '팬티',
  'last': True},
 {'wholeCategoryName': '패션의류>남성언더웨어/잠옷>러닝',
  'id': '50000846',
  'name': '러닝',
  'last': True},
 {'wholeCategoryName': '패션의류>남성언더웨어/잠옷>러닝팬티세트',
  'id': '50000847',
  'name': '러닝팬티세트',
  'last': True},
 {'wholeCategoryName': '패션의류>남성언더웨어/잠옷>잠옷/홈웨어',
  'id': '50000848',
  'name': '잠옷/홈웨어',
  'last': True},
 {'wholeCategoryName': '패션의류>남성언더웨어/잠옷>시즌성내의',
  'id': '50001456',
  'name': '시즌성내의',
  'last': False},
 {'wholeCategoryName': '패션의류>남성언더웨어/잠옷>보정속옷',
  'id': '50001455',
  'name': '보정속옷',
  'last': False}]

## 브랜드 조회

In [3]:
find_brand_id('나이키')

[{'id': 1269, 'name': '나이키'},
 {'id': 34544, 'name': '나이키골프'},
 {'id': 3404692, 'name': '나이키스윔'},
 {'id': 20889, 'name': '나이키키즈'}]

## 제조사 조회

In [7]:
find_manufacturers_id('나이키')

[{'id': 259842, 'name': '나이키'}]

## 카탈로그 조회

In [8]:
find_catalog_list('나이키 에어맥스97')

[나이키 에어맥스97] 검색 성공 : 7건


{'contents': [{'wholeCategoryName': '패션잡화>남성신발>운동화>러닝화',
   'categoryId': '50003854',
   'manufacturerCode': 259842,
   'manufacturerName': '나이키',
   'brandCode': 1269,
   'brandName': '나이키',
   'id': 31092012848,
   'name': '에어맥스 97 921826-101'},
  {'wholeCategoryName': '패션잡화>남성신발>운동화>러닝화',
   'categoryId': '50003854',
   'manufacturerCode': 259842,
   'manufacturerName': '나이키',
   'brandCode': 1269,
   'brandName': '나이키',
   'id': 43315002627,
   'name': '에어맥스 97 921826 019'},
  {'wholeCategoryName': '패션잡화>남성신발>운동화>러닝화',
   'categoryId': '50003854',
   'manufacturerCode': 259842,
   'manufacturerName': '나이키',
   'brandCode': 1269,
   'brandName': '나이키',
   'id': 45606957618,
   'name': '에어맥스 97 921826 203'},
  {'wholeCategoryName': '패션잡화>남성신발>운동화>러닝화',
   'categoryId': '50003854',
   'manufacturerCode': 259842,
   'manufacturerName': '나이키',
   'brandCode': 1269,
   'brandName': '나이키',
   'id': 19472129052,
   'name': '에어 맥스 97 921826-001'},
  {'wholeCategoryName': '패션잡화>남성신발>운동화>러닝화'

In [9]:
get_catalog_record(19472129052)

{'wholeCategoryName': '패션잡화>남성신발>운동화>러닝화',
 'categoryId': '50003854',
 'manufacturerCode': 259842,
 'manufacturerName': '나이키',
 'brandCode': 1269,
 'brandName': '나이키',
 'id': 19472129052,
 'name': '에어 맥스 97 921826-001'}

## 원산지 코드 정보

In [10]:
get_originAreaCode('일본')

'0200036'

## 이미지 등록

In [11]:
image_folder_path = 'image/240119/'

image_upload(image_folder_path)

이미지 업로드 실패


# 상품등록

### 상품정보등록고시

In [12]:
# 상품정보등록고시는 로컬에 한번 저장하고, 업데이트 함수를 만들어서 종종 업데이트 하면 됨.
Update_ItemInfoNotification()

*** 상품정보 제공공시 master 업데이트 완료 (ItemInfo_notification.json) ***


In [4]:
get_ItemInfo_notification_Categories()

*** 상품정보 제공공시 카테고리 리스트 가져오기 성공 ***


[('의류', 'WEAR'),
 ('구두/신발', 'SHOES'),
 ('가방', 'BAG'),
 ('패션잡화(모자/벨트/액세서리 등)', 'FASHION_ITEMS'),
 ('침구류/커튼', 'SLEEPING_GEAR'),
 ('가구(침대/소파/싱크대/DIY제품 등)', 'FURNITURE'),
 ('영상가전(TV류)', 'IMAGE_APPLIANCES'),
 ('가정용전기제품(냉장고/세탁기/식기세척기/전자레인지 등)', 'HOME_APPLIANCES'),
 ('계절가전(에어컨/온풍기 등)', 'SEASON_APPLIANCES'),
 ('사무용기기(컴퓨터/노트북/프린터 등)', 'OFFICE_APPLIANCES'),
 ('광학기기(디지털카메라/캠코더 등)', 'OPTICS_APPLIANCES'),
 ('소형전자(MP3/전자사전 등)', 'MICROELECTRONICS'),
 ('내비게이션', 'NAVIGATION'),
 ('자동차용품 (자동차부품/기타 자동차용품 등)', 'CAR_ARTICLES'),
 ('의료기기', 'MEDICAL_APPLIANCES'),
 ('주방용품', 'KITCHEN_UTENSILS'),
 ('화장품', 'COSMETIC'),
 ('귀금속/보석/시계류', 'JEWELLERY'),
 ('식품(농.축.수산물)', 'FOOD'),
 ('가공식품', 'GENERAL_FOOD'),
 ('건강기능식품', 'DIET_FOOD'),
 ('어린이제품', 'KIDS'),
 ('악기', 'MUSICAL_INSTRUMENT'),
 ('스포츠용품', 'SPORTS_EQUIPMENT'),
 ('서적', 'BOOKS'),
 ('물품대여서비스(서적,유아용품,행사용품 등)', 'RENTAL_ETC'),
 ('디지털콘텐츠(음원,게임,인터넷강의 등)', 'DIGITAL_CONTENTS'),
 ('상품권/쿠폰', 'GIFT_CARD'),
 ('모바일쿠폰', 'MOBILE_COUPON'),
 ('영화/공연', 'MOVIE_SHOW'),
 ('기타 용역', 'ETC_SER

In [20]:
# 먼저 카테고리 선택
my_category = 'SHOES'

In [21]:
get_ItemInfo_notifications('SHOES')

*** SHOES 상품정보 제공공시 항목 가져오기 성공 ***


[('material', '제품의 주 소재'),
 ('color', '색상'),
 ('size', '발길이'),
 ('height', '굽높이'),
 ('manufacturer', '제조자(사)'),
 ('caution', '취급 시 주의사항'),
 ('warrantyPolicy', '품질 보증 기준'),
 ('afterServiceDirector', 'A/S 책임자와 전화번호')]

In [22]:
get_ItemInfo_notifications('WEAR')


*** WEAR 상품정보 제공공시 항목 가져오기 성공 ***


[('material', '제품 소재'),
 ('color', '색상'),
 ('size', '치수'),
 ('manufacturer', '제조자(사)'),
 ('caution', '세탁 방법 및 취급 시 주의사항'),
 ('packDate', '제조연월'),
 ('packDateText', '제조연월 직접 입력'),
 ('warrantyPolicy', '품질 보증 기준'),
 ('afterServiceDirector', 'A/S 책임자와 전화번호')]

In [23]:
productInfoProvidedNotice = {

}

def lower_string(input_string):
    words = input_string.lower().split('_')
    result = words[0] + ''.join(word.capitalize() for word in words[1:])
    return result

productInfoProvidedNotice['productInfoProvidedNoticeType'] = my_category
productInfoProvidedNotice[lower_string(my_category)] = {
        # 고정 : returnCostReason, noRefundReason, qualityAssuranceStandard, compensationProcedure, troubleShootingContents -> 1: 상품상세 참조
        "returnCostReason": "1", 
        "noRefundReason": "1",
        "qualityAssuranceStandard": "1",
        "compensationProcedure": "1",
        "troubleShootingContents": "1",
    }

# 각 카테고리에 필요한 항목들 입력
productInfoProvidedNotice[lower_string(my_category)]["material"] = "인조 가죽"
productInfoProvidedNotice[lower_string(my_category)]["color"] = "블랙, 화이트, 옐로우, 레드"
productInfoProvidedNotice[lower_string(my_category)]["size"] = "245~280"
productInfoProvidedNotice[lower_string(my_category)]["manufacturer"] = "아식스"
productInfoProvidedNotice[lower_string(my_category)]["caution"] = "상세설명 참조"
productInfoProvidedNotice[lower_string(my_category)]["warrantyPolicy"] = "상세설명 참조"
productInfoProvidedNotice[lower_string(my_category)]["afterServiceDirector"] = "상세설명 참조"



In [24]:
productInfoProvidedNotice

{'productInfoProvidedNoticeType': 'SHOES',
 'shoes': {'returnCostReason': '1',
  'noRefundReason': '1',
  'qualityAssuranceStandard': '1',
  'compensationProcedure': '1',
  'troubleShootingContents': '1',
  'material': '인조 가죽',
  'color': '블랙, 화이트, 옐로우, 레드',
  'size': '245~280',
  'manufacturer': '아식스',
  'caution': '상세설명 참조',
  'warrantyPolicy': '상세설명 참조',
  'afterServiceDirector': '상세설명 참조'}}

### 고정 변수

In [4]:
get_originAreaCode('일본')

'0200036'

In [5]:
# AS 정보
afterServiceTelephoneNumber = '010-2936-1595'
afterServiceGuideContent = '상품 오배송 및 파송시 A/S 도와드리겠습니다'

# 판매 기간 정보
saleStartDate = (datetime.datetime.now() + datetime.timedelta(minutes=3)).strftime('%Y-%m-%dT%H:%M:%SZ')  # 현재로부터 3분 뒤
saleEndDate = '2099-01-13T01:30:56Z'

# 세일 기간 정보
# 세일 기간 정보
Discount_StartDate = (datetime.datetime.now() + datetime.timedelta(hours=1)).strftime('%Y-%m-%dT%H')+':00:00Z'  # 현재로부터 다음 정각
Discount_EndDate = '2099-01-13T01:59:00Z'


# 원산지 정보
originAreaInfo_JAPAN = {
"originAreaCode": get_originAreaCode('일본'),
"importer": "japan", # 수입사인 경우 필수
"content": "string",
"plural": True
}

# 부가가치세 타입 : 디폴트는 과세상품 -> TAX(과세 상품), DUTYFREE(면세 상품), SMALL(영세 상품)
taxType =  "TAX"


#### 입력필요변수

In [6]:
# 상품이름
name = '아식스 안전화 윈잡 보아 안전 작업화 CP306 1273A029'           
# 상품 상세설명
detailContent = '아식스 안전화 윈잡 보아 안전 작업화 CP306 1273A029 입니다.'

# 판매 가격 (필수)
salePrice = 150000 
# 재고량
stockQuantity = 100



In [7]:
# 내가 자주 사용하는 카탈로그 dictonary로 만들 필요 있음
find_catalog_list('아식스 안전화 306')  # 카탈로그 확인해서 catalog_Id만 확정하면 나머지 다 따라옴

[아식스 안전화 306] 검색 성공 : 7건


{'contents': [{'wholeCategoryName': '패션잡화>남성신발>기능화>작업화/안전화',
   'categoryId': '50003860',
   'manufacturerCode': 0,
   'manufacturerName': '',
   'brandName': '',
   'id': 41899084761,
   'name': '아식스 안전화 CP306'},
  {'wholeCategoryName': '패션잡화>남성신발>기능화>작업화/안전화',
   'categoryId': '50003860',
   'manufacturerCode': 20048,
   'manufacturerName': '아식스',
   'brandCode': 654,
   'brandName': '아식스',
   'id': 42538144516,
   'name': '아식스 안전화 윈잡 보아 안전 작업화 CP306 1273A029'},
  {'wholeCategoryName': '패션잡화>남성신발>기능화>작업화/안전화',
   'categoryId': '50003860',
   'manufacturerCode': 20048,
   'manufacturerName': '아식스',
   'brandCode': 654,
   'brandName': '아식스',
   'id': 42732149737,
   'name': '아식스 안전화 윈잡 보아 남여공용 작업화 FCP306 1273A087'},
  {'wholeCategoryName': '패션잡화>남성신발>기능화>작업화/안전화',
   'categoryId': '50003860',
   'manufacturerCode': 20048,
   'manufacturerName': '아식스',
   'brandCode': 654,
   'brandName': '아식스',
   'id': 38893484907,
   'name': '아식스 안전화 보아 마그마 윈잡 작업화 세이츠티 슈즈 F FCP306'},
  {'wholeCatego

In [8]:
# https://www.amazon.co.jp/%E3%82%A2%E3%82%B7%E3%83%83%E3%82%AF%E3%82%B9-%E3%83%AF%E3%83%BC%E3%82%AD%E3%83%B3%E3%82%B0-%E3%82%A6%E3%82%A3%E3%83%B3%E3%82%B8%E3%83%A7%E3%83%96-CP306-fuzeGEL%E6%90%AD%E8%BC%89/dp/B081JSHJT8/ref=sr_1_41?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&crid=I12PM0M11J21&keywords=boa&qid=1705930553&sprefix=boa%2Caps%2C207&sr=8-41

In [9]:
# 카테고리 id를 먼저 지정해야함 -> 카탈로그로 인해 나중에 바뀔수도 있지만.
leafCategoryId = ''




# 카탈로그 id 활용해서 얻을 수 있음.  ** 없는 경우: 직접 입력    
catalog_Id = '42538144516'

recode = get_catalog_record(catalog_Id)
leafCategoryId = recode['categoryId']             
catalog_Name = recode['name']
manufacturerName =  recode['manufacturerName']      # find_manufacturers_id('나이키') # 활용 -> 존재하는지 여부 확인
brandId = recode['brandCode']                       # find_brand_id('나이키') # 활용  -> 브랜드 ID 확인
brandName = recode['brandName']    


naverShoppingSearchInfo = {
        "modelId": catalog_Id,
        "modelName": catalog_Name,
        "manufacturerName": manufacturerName,
        "brandId": brandId,
        "brandName": brandName
      }
    


In [10]:
# options_and_urls = image_upload('image/240119/option/')
# options_size = [ str(225+ n*5) for n in range(16) ]       # 사이즈
# options_color = [x[0] for x in options_and_urls]



*** 이미지 업로드 성공 ***


In [11]:
get_standard_attribute(leafCategoryId, '사이즈(한국)',  options_size )     

*** [leafCategoryId: 50003860] 카테고리의 사이즈(한국) 표준 옵션 가져오기 성공 ***


[{'attributeId': 10014204,
  'attributeValueId': 10696136,
  'attributeValueName': '225'},
 {'attributeId': 10014204,
  'attributeValueId': 10696137,
  'attributeValueName': '230'},
 {'attributeId': 10014204,
  'attributeValueId': 10696138,
  'attributeValueName': '235'},
 {'attributeId': 10014204,
  'attributeValueId': 10696139,
  'attributeValueName': '240'},
 {'attributeId': 10014204,
  'attributeValueId': 10696140,
  'attributeValueName': '245'},
 {'attributeId': 10014204,
  'attributeValueId': 10696141,
  'attributeValueName': '250'},
 {'attributeId': 10014204,
  'attributeValueId': 10696142,
  'attributeValueName': '255'},
 {'attributeId': 10014204,
  'attributeValueId': 10696143,
  'attributeValueName': '260'},
 {'attributeId': 10014204,
  'attributeValueId': 10696144,
  'attributeValueName': '265'},
 {'attributeId': 10014204,
  'attributeValueId': 10696145,
  'attributeValueName': '270'},
 {'attributeId': 10014204,
  'attributeValueId': 10696146,
  'attributeValueName': '275'},

In [12]:
get_standard_attribute(leafCategoryId, '색상',  options_color )     

*** [leafCategoryId: 50003860] 카테고리의 색상 표준 옵션 가져오기 성공 ***


[{'attributeId': 10014203,
  'attributeValueId': 10031739,
  'attributeValueName': '레드',
  'attributeColorCode': '#FF0000'},
 {'attributeId': 10014203,
  'attributeValueId': 10031742,
  'attributeValueName': '옐로우',
  'attributeColorCode': '#FFFF00'},
 {'attributeId': 10014203,
  'attributeValueId': 10531869,
  'attributeValueName': '화이트',
  'attributeColorCode': 'color-type-white'},
 {'attributeId': 10014203,
  'attributeValueId': 10031727,
  'attributeValueName': '블랙',
  'attributeColorCode': '#000000'}]

신발 옵션

In [13]:
# 이미지 등록 
image_urls = image_upload('image/240119/option/')

# 대표이미지 URL
representativeImageURL = image_urls[0][1] # 블랙
# 추가 이미지 URL 목록
optionalImagesURLs = [ {"url": x[1]} for x in image_urls[1:] ]


# 옵션 이미지 등록 
options_and_urls = image_upload('image/240119/option/')
options_size = [ str(225+ n*5) for n in range(16) ]       # 사이즈
options_color = [x[0] for x in options_and_urls]

standard_attribute_size = get_standard_attribute(leafCategoryId, '사이즈(한국)',  options_size )     
standard_attribute_color = get_standard_attribute(leafCategoryId, '색상',  options_color )     


OptionGroup1 =  {
        "groupName" : "색상",
        "standardOptionAttributes" : [ 
            {
             "attributeId": color['attributeId'],
             "attributeValueId": color['attributeValueId'],
             "attributeValueName": color['attributeValueName'],
             "imageUrls": options_and_urls[idx][1]
             }
            for idx, color in enumerate(standard_attribute_color)
        ]
      }

OptionGroup2 = {
        "groupName" : "사이즈",
        "standardOptionAttributes" : [
            {
             "attributeId": size['attributeId'],
             "attributeValueId": size['attributeValueId'],
             "attributeValueName": size['attributeValueName'],
             }
             for idx, size in enumerate(standard_attribute_size)
             
        ]
      }


*** 이미지 업로드 성공 ***
*** 이미지 업로드 성공 ***
*** [leafCategoryId: 50003860] 카테고리의 사이즈(한국) 표준 옵션 가져오기 성공 ***
*** [leafCategoryId: 50003860] 카테고리의 색상 표준 옵션 가져오기 성공 ***


In [14]:
OptionGroup1

{'groupName': '색상',
 'standardOptionAttributes': [{'attributeId': 10014203,
   'attributeValueId': 10031739,
   'attributeValueName': '레드',
   'imageUrls': 'http://shop1.phinf.naver.net/20240122_142/1705929358409wTYVQ_JPEG/5455576275694206_1220525852.jpg'},
  {'attributeId': 10014203,
   'attributeValueId': 10031742,
   'attributeValueName': '옐로우',
   'imageUrls': 'http://shop1.phinf.naver.net/20240122_136/1705923655839s4Geh_JPEG/31112929740018466_1265233498.jpg'},
  {'attributeId': 10014203,
   'attributeValueId': 10531869,
   'attributeValueName': '화이트',
   'imageUrls': 'http://shop1.phinf.naver.net/20240122_150/17059293589071DlnP_JPEG/56280742860303359_1054031388.jpg'},
  {'attributeId': 10014203,
   'attributeValueId': 10031727,
   'attributeValueName': '블랙',
   'imageUrls': 'http://shop1.phinf.naver.net/20240122_112/1705929358680v11dv_JPEG/56541127863940465_1501463376.jpg'}]}

In [15]:
items = [[x[0] for x in options_and_urls], options_size]

# 옵션 매칭
optionStandards =  [
              {
                    "id": idx+1,
                    "optionName1": f"{combi[0]}",
                    "optionName2": f"{combi[1]}",
                    "stockQuantity": 9999,
                    "usable": True
                }
                for idx, combi in enumerate(list(product(*items)))
                
  ] 


# 조합형으로 수정 필요
simpleOption_template = { 
    "simpleOptionSortType": "CREATE",
    "optionSimple": [ ],
    "optionCombinationSortType": "CREATE",
    # 표준형 옵션 그룹 : 그룹 네임과 옵션 속성들 
    "standardOptionGroups" : [
     OptionGroup1,
     OptionGroup2
    ],
    "optionStandards":  optionStandards,

    "useStockManagement": False                    # 옵션 재고 수량 관리 사용 여부
}

optionInfo = simpleOption_template

In [16]:
# # 구매수량 
# purchaseQuantityInfo =  {
#     "minPurchaseQuantity": 1,           # 최소 구매 수량
#     "maxPurchaseQuantityPerId": 100,    # 1인 최대 구매수량
#     "maxPurchaseQuantityPerOrder": 100  # 1회 최대 구매수량
# }


# 제조일자 : 인증 유형이 방송통신기자재 적합인증/적합등록/잠정인증인 경우 필수. 'yyyy-MM-dd' 형식 입력.
manufactureDate =  '2023-11-01' # "yyyy-mm-dd" 형태

# # 출시일자: 브랜드 스토어 한정, 수정불가
# releaseDate = '' # "yyyy-mm-dd" 형태

# 유효일자: 
validDate = '2033-11-01'   # "yyyy-mm-dd" 형태




productCertificationInfos = [
        # {
        #   "certificationInfoId": 0,                        # 인증유형
        #   "certificationKindType": "OVERSEAS",             # 인증정보종류코드 : KC_CERTIFICATION(KC 인증), CHILD_CERTIFICATION(어린이제품 인증), GREEN_PRODUCTS(친환경 인증), OVERSEAS(구매대행(구매대행 선택 시 인증 정보 필수 등록)), PARALLEL_IMPORT(병행수입(병행수입 선택 시 인증 정보 필수 등록)), ETC(기타 인증)
        #   "name": "string",                                # 인증기관명
        #   "certificationNumber": "string",                 # 인증번호
        #   "certificationMark": True,                       # 인증마크 사용 여부 - 디폴트는 false
        #   "companyName": "string",                         # 인증 상호명
        #   "certificationDate": "2024-01-13"                # 인증 일자
        # }
      ]

certificationTargetExcludeContent =  {
    "childCertifiedProductExclusionYn": True,                     # 어린이제품 인증 대상 제외 여부: 필수. 미입력 시 false로 저장됩니다
    "kcExemptionType": "OVERSEAS",                                # KC 면제 대상 타입  (구매대행/병행이면 필수 입력): SAFE_CRITERION(안전기준준수대상(안전기준준수대상 예외 카테고리가 아닌 경우에도 설정 가능, 식품 카테고리 외)), OVERSEAS(구매대행), PARALLEL_IMPORT(병행수입)
    "kcCertifiedProductExclusionYn": "KC_EXEMPTION_OBJECT",       # TRUE(KC 인증 대상 아님), FALSE(KC 인증 대상), KC_EXEMPTION_OBJECT(안전기준준수, 구매대행, 병행수입인 경우 필수 입력)
    "greenCertifiedProductExclusionYn": True                      # 친환경 인증 대상 제외 여부
  },

# 미성년자 사용 가능 여부
minorPurchasable =  True

# 상품정보제공고시
productInfoProvidedNoticeType = 'WEAR'




# 문화비 소득공제 여부: 예외 카테고리 중 도서_일반, 도서_해외, 도서_중고, 도서_E북, 도서_오디오북, 문화비_소득공제에 해당하는 경우 입력할 수 있습니다. 미입력 시 false로 저장됩니다.
cultureCostIncomeDeductionYn = False

# 맞춤 제작 상품 여부
customProductYn = False
# 자체 제작 상품 여부
itselfProductionProductYn =  False
# 브랜드 인증 여부
# brandCertificationYn = True
# (SEO(Search engine optimization) 정보)
# pageTitle/metaDescription : 상품 정보를 SNS 등 소셜 서비스에 공유 시 검색 최적화를 위한 기능으로 검색에 활용될 수 있도록 설정할 수 있습니다.
pageTitle = "title태그"
metaDescription = "한줄 설명"
tags = []

seoInfo = {
  "pageTitle": pageTitle,
  "metaDescription": metaDescription,
  "sellerTags": [{ "code": idx, "text": tag } for idx,tag in enumerate(tags)]
}


discount_unitType = 'PERCENT' # 할인 단위 타입 ->  PERCENT, WON만 입력 가능합니다.
discount_value  = '10'        # 할인 단위에 따른 값을 입력합니다.
discount_startDate = saleStartDate
discount_endDate = saleEndDate

In [17]:
deliveryInfo = {
      "deliveryType": "DELIVERY",                  # DELIVERY(택배, 소포, 등기), DIRECT(직접배송(화물배달))
      "deliveryAttributeType": "NORMAL",
      "deliveryCompany": "EPOST",
      "deliveryBundleGroupUsable": True,
      # "deliveryBundleGroupId": 0,
      "deliveryFee": {
        "deliveryFeeType": "PAID",
        "baseFee": 4000,
        "deliveryFeePayType": "PREPAID",
        "deliveryFeeByArea": {
          "deliveryAreaType": "AREA_2",
          "area2extraFee": 5000,
        },
      },

      "claimDeliveryInfo": {
        "returnDeliveryCompanyPriorityType": "PRIMARY",
        "returnDeliveryFee": 4000,
        "exchangeDeliveryFee": 8000,
      },
      # "installationFee": False, # 별도 설치비 유무
      # "expectedDeliveryPeriodType": "ETC",
      # "expectedDeliveryPeriodDirectInput": "string",
      # "todayStockQuantity": 0,
      # "customProductAfterOrderYn": True,
      # "hopeDeliveryGroupId": 0
    }

In [27]:
post_body = { 
    
  ########################################
  ### 1. 원상품 정보 구조체             ###
  ########################################
  "originProduct": {
      
    # 상품 상태: 상품 등록시엔 SALE만 입력 가능
    "statusType": "SALE",       

    # 새상품(NEW) / 중고상품 (OLD) 
    "saleType": "NEW",           
    
    # 카테고리 ID
    "leafCategoryId": leafCategoryId,  
    
    # 상품명
    "name": name,            
    
    # 상품 상세설명
    "detailContent": detailContent,   

    # 상품 이미지로 대표 이미지(1000x1000픽셀 권장)와 최대 9개의 추가 이미지 목록을 제공할 수 있습니다. 
    # 대표 이미지는 필수이고 추가 이미지는 선택 사항입니다. 
    # 이미지 URL은 반드시 상품 이미지 다건 등록 API로 이미지를 업로드하고 반환받은 URL 값을 입력해야 합니다.
    "images": {
      "representativeImage": { "url": representativeImageURL },  # 대표이미지
      "optionalImages": optionalImagesURLs      # 추가이미지
    },

    # 판매 시작일, 판매 마감일
      # 'yyyy-MM-dd'T'HH:mm[:ss][.SSS]XXX' 형식으로 입력
    "saleStartDate": saleStartDate,  # 현재부터 3분 후      
    "saleEndDate": saleEndDate,       # 2099-01-13T01:30:56Z,    

    # 판매 가격
    "salePrice": salePrice,

    # 재고량                              
    "stockQuantity": 0,                           
    
    # 배달정보 : 이건 유형을 만들어서 넣는 방식으로 하자. -> 언제 다 넣냐..
    "deliveryInfo": deliveryInfo,

    ## Array of objects (물류사 정보): 풀필먼트 이용 상품은 필수 -> 네이버 물류 연합군의 풀필먼트 서비스를 이용하고 있는 경우 상품에 물류사/물류센터 정보를 입력합니다.
    # "productLogistics": [
    #   { "logisticsCompanyId": "string"}            # string (물류사 ID)
    # ],

    # 원상품 상세 속성
    "detailAttribute": {
      # 네이버쇼핑 검색어 정보
      # "naverShoppingSearchInfo": {
      #   "modelId": catalog_Id,
      #   "modelName": catalog_Name,
      #   "manufacturerName": manufacturerName,
      #   "brandId": brandId,
      #   "brandName": brandName
      # },
      # A/S 정보
      "afterServiceInfo": {
        "afterServiceTelephoneNumber": afterServiceTelephoneNumber,
        "afterServiceGuideContent": afterServiceGuideContent
      },
      # 구매 수량 정보
      # "purchaseQuantityInfo": purchaseQuantityInfo,
      # 원산지 정보
      "originAreaInfo": originAreaInfo_JAPAN,
      
      # # 판매자 코드 정보
      # "sellerCodeInfo": {
      #   "sellerManagementCode": "string",
      #   "sellerBarcode": "string",
      #   "sellerCustomCode1": "string",
      #   "sellerCustomCode2": "string"
      # },

      # 옵션 정보. [단독형 옵션, 조합형 옵션, 직접 입력형] 옵션 중 최소 한 개는 입력해야 함.
      # 즉, optionSimple, optionCustom, optionCombinations 중 하나는 입력해야 함.
      # 단독형 옵션과 조합형 옵션은 함께 사용할 수 없다.
      "optionInfo": optionInfo,


      # 추가 상품 정보: 등록된 기본 상품과 함께 구성하면 좋은 상품으로 구성 : 
      # 여긴 디폴트로 생략하고, 네이버 스토에서 수동 설정하는걸로 하자.
      # "supplementProductInfo": {
      #   "sortType": "CREATE",
      #   "supplementProducts": [
      #     {
      #       "id": 0,
      #       "groupName": "string",
      #       "name": "string",
      #       "price": 0,
      #       "stockQuantity": 0,
      #       "sellerManagementCode": "string",
      #       "usable": true
      #     }
      #   ]
      # },

      #리뷰 노출 설정 정보 
      "purchaseReviewInfo": {
        "purchaseReviewExposure": True,
        # "reviewUnExposeReason": "string"  # 리뷰 미노출 사유. 리뷰 노출 여부가 false일 경우 리뷰 미노출 사유를 입력해야 합니다.
      },

      ## ISBN 정보
      # "isbnInfo": {
      #   "isbn13": "string",
      #   "issn": "string",
      #   "independentPublicationYn": True
      # },
      ## 도서 항목 부가 정보
      # "bookInfo": {
      #   "publishDay": "string",
      #   "publisher": { "code": "string", "text": "string" },
      #   "authors": [{ "code": "string", "text": "string" }],
      #   "illustrators": [{ "code": "string", "text": "string" }],
      #   "translators": [{ "code": "string", "text": "string" }]
      # },

      # # (이벤트 문구(홍보 문구 대체))
      # "eventPhraseCont": eventPhraseCont,
      # 제조일자 : 인증 유형이 방송통신기자재 적합인증/적합등록/잠정인증인 경우 필수. 'yyyy-MM-dd' 형식 입력.
      "manufactureDate": manufactureDate, # "2024-01-13",
      # # 출시일자 : 수정/삭제 불가능. 브랜드스토어 한정
      # "releaseDate": releaseDate,
      # 유효일자: 
      "validDate": validDate,
      # 부가가치세 타입 : 디폴트는 과세상품 -> TAX(과세 상품), DUTYFREE(면세 상품), SMALL(영세 상품)
      "taxType": "TAX",   # taxType,
      
      # 인증정보 목록:  '어린이제품 인증 대상' 카테고리 상품인 경우 필수
      "productCertificationInfos": productCertificationInfos,
      # 인증대상 제외여부 검토
      "certificationTargetExcludeContent": {
    "childCertifiedProductExclusionYn": True,                     # 어린이제품 인증 대상 제외 여부: 필수. 미입력 시 false로 저장됩니다
    "kcExemptionType": "OVERSEAS",                                # KC 면제 대상 타입  (구매대행/병행이면 필수 입력): SAFE_CRITERION(안전기준준수대상(안전기준준수대상 예외 카테고리가 아닌 경우에도 설정 가능, 식품 카테고리 외)), OVERSEAS(구매대행), PARALLEL_IMPORT(병행수입)
    "kcCertifiedProductExclusionYn": "KC_EXEMPTION_OBJECT",       # TRUE(KC 인증 대상 아님), FALSE(KC 인증 대상), KC_EXEMPTION_OBJECT(안전기준준수, 구매대행, 병행수입인 경우 필수 입력)
    "greenCertifiedProductExclusionYn": True                      # 친환경 인증 대상 제외 여부
  },

      # # 판매자 특이사항
      # "sellerCommentContent": "없음",
      # # 판매자 특이사항 사용 여부
      # "sellerCommentUsable": False,

      # 미성년자 구매 가능 여부
      "minorPurchasable": minorPurchasable,

      # # E 쿠폰
      # "ecoupon": {
      #   "periodType": "FIXED",
      #   "validStartDate": "2024-01-13",
      #   "validEndDate": "2024-01-13",
      #   "periodDays": 0,
      #   "publicInformationContents": "string",
      #   "contactInformationContents": "string",
      #   "usePlaceType": "PLACE",
      #   "usePlaceContents": "string",
      #   "restrictCart": True,
      #   "siteName": "string"
      # },

      # 상품정보제공고시
      "productInfoProvidedNotice": productInfoProvidedNotice,

  
      # 상품속성 목록
      "productAttributes": [
        # {
        #   "attributeSeq": 0,
        #   "attributeValueSeq": 0,
        #   "attributeRealValue": "string",
        #   "attributeRealValueUnitCode": "string"
        # }
      ],
      # 문화비 소득공제 여부 : 예외 카테고리 중 도서_일반, 도서_해외, 도서_중고, 도서_E북, 도서_오디오북, 문화비_소득공제에 해당하는 경우 입력할 수 있습니다. 미입력 시 false로 저장됩니다.
      # "cultureCostIncomeDeductionYn": cultureCostIncomeDeductionYn,
      # 맞춤 제작 상품 여부
      "customProductYn": customProductYn,
      # 자체 제작 상품 여부
      "itselfProductionProductYn": itselfProductionProductYn,
      # 브랜드 인증 여부
      # "brandCertificationYn": brandCertificationYn,
      # (SEO(Search engine optimization) 정보)
      "seoInfo": seoInfo
    },
 
    # 상품 고객 헤택 정보
    "customerBenefit": {
      # 즉시 할인 정책
      "immediateDiscountPolicy": {
        # PC 할인 방법
        "discountMethod": {
          "value": discount_value,
          "unitType": discount_unitType,
          "startDate": Discount_StartDate,
          "endDate": Discount_EndDate      # '2099-01-13T01:30:56Z'
        },

        # 모바일 할인방법
        "mobileDiscountMethod": {
          "value": discount_value,
          "unitType": discount_unitType,
          "startDate": Discount_StartDate,
          "endDate": Discount_EndDate      # '2099-01-13T01:30:56Z'
        }
      },
      # "purchasePointPolicy": {
      #   "value": 0,
      #   "unitType": "PERCENT",
      #   "startDate": "2024-01-13",
      #   "endDate": "2024-01-13"
      # },
      # "reviewPointPolicy": {
      #   "textReviewPoint": 0,
      #   "photoVideoReviewPoint": 0,
      #   "afterUseTextReviewPoint": 0,
      #   "afterUsePhotoVideoReviewPoint": 0,
      #   "storeMemberReviewPoint": 0,
      #   "startDate": "2024-01-13",
      #   "endDate": "2024-01-13"
      # },
      # "freeInterestPolicy": {
      #   "value": 0,
      #   "startDate": "2024-01-13",
      #   "endDate": "2024-01-13"
      # },
      # "giftPolicy": { "presentContent": "string" },
      # "multiPurchaseDiscountPolicy": {
      #   "discountMethod": {
      #     "value": 1,
      #     "unitType": "PERCENT",
      #     "startDate": "2024-01-13",
      #     "endDate": "2024-01-13"
      #   },
      #   "orderValue": 0,
      #   "orderValueUnitType": "PERCENT"
      # },
      # "reservedDiscountPolicy": {
      #   "discountMethod": {
      #     "value": 1,
      #     "unitType": "PERCENT",
      #     "startDate": "2024-01-13T01:30:56Z",
      #     "endDate": "2024-01-13T01:30:56Z"
      #   }
      # }
    }
  },
  
  ########################################
  ### 2 스마트스토어 채널상품 정보 구조체 ###
  ########################################
  "smartstoreChannelProduct": {
    # "channelProductName": "string",
    # "bbsSeq": 0,
    # "storeKeepExclusiveProduct": True,
    "naverShoppingRegistration": False,      # 네이버 쇼핑 광고주가 아닌 경우에는 false로 저장됩니다.
    "channelProductDisplayStatusType": "ON"  # WAIT(전시 대기), ON(전시 중), SUSPENSION(전시 중지)
  },
  # "windowChannelProduct": {
  #   # "channelProductName": "string",
  #   # "bbsSeq": 0,
  #   # "storeKeepExclusiveProduct": True,
  #   "naverShoppingRegistration": False,
  #   # "channelNo": 0,
  #   "best": True
  # }
}




In [28]:
# 필수가 아니면 이렇게 따로 넣자.
post_body['originProduct']['detailAttribute']['naverShoppingSearchInfo'] = naverShoppingSearchInfo

In [35]:
Discount_EndDate+'+09:00'

'2099-01-13T01:59:00Z+09:00'

In [36]:
type(post_body)

dict

In [37]:
with open('postbody.json', 'w') as f:
    json.dump(json.dumps(post_body), f) 

In [38]:
len(json.load(open('postbody.json', 'r')))

13023

In [39]:
import http.client

conn = http.client.HTTPSConnection("api.commerce.naver.com")

payload = json.load(open('postbody.json', 'r'))
headers = {
    'Authorization': f"Bearer {access_token}",
    'content-type': "application/json"
    }

conn.request("POST", "/external/v2/products", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"originProductNo":9895008206,"smartstoreChannelProductNo":9942583717}


In [47]:
payload

'{"originProduct": {"statusType": "SALE", "saleType": "NEW", "leafCategoryId": "50003860", "name": "\\uc544\\uc2dd\\uc2a4 \\uc548\\uc804\\ud654 \\uc708\\uc7a1 \\ubcf4\\uc544 \\uc548\\uc804 \\uc791\\uc5c5\\ud654 CP306 1273A029", "detailContent": "\\uc544\\uc2dd\\uc2a4 \\uc548\\uc804\\ud654 \\uc708\\uc7a1 \\ubcf4\\uc544 \\uc548\\uc804 \\uc791\\uc5c5\\ud654 CP306 1273A029 \\uc785\\ub2c8\\ub2e4.", "images": {"representativeImage": {"url": "http://shop1.phinf.naver.net/20240122_142/1705929358409wTYVQ_JPEG/5455576275694206_1220525852.jpg"}, "optionalImages": [{"url": "http://shop1.phinf.naver.net/20240122_136/1705923655839s4Geh_JPEG/31112929740018466_1265233498.jpg"}, {"url": "http://shop1.phinf.naver.net/20240122_150/17059293589071DlnP_JPEG/56280742860303359_1054031388.jpg"}, {"url": "http://shop1.phinf.naver.net/20240122_112/1705929358680v11dv_JPEG/56541127863940465_1501463376.jpg"}]}, "saleStartDate": "2024-02-12T13:58:02Z", "saleEndDate": "2099-01-13T01:30:56Z", "salePrice": 150000, "sto

In [33]:
json.dumps(post_body)

'{"originProduct": {"statusType": "SALE", "saleType": "NEW", "leafCategoryId": "50003860", "name": "\\uc544\\uc2dd\\uc2a4 \\uc548\\uc804\\ud654 \\uc708\\uc7a1 \\ubcf4\\uc544 \\uc548\\uc804 \\uc791\\uc5c5\\ud654 CP306 1273A029", "detailContent": "\\uc544\\uc2dd\\uc2a4 \\uc548\\uc804\\ud654 \\uc708\\uc7a1 \\ubcf4\\uc544 \\uc548\\uc804 \\uc791\\uc5c5\\ud654 CP306 1273A029 \\uc785\\ub2c8\\ub2e4.", "images": {"representativeImage": {"url": "http://shop1.phinf.naver.net/20240122_136/1705923655839s4Geh_JPEG/31112929740018466_1265233498.jpg"}, "optionalImages": [{"url": "http://shop1.phinf.naver.net/20240122_136/1705923655839s4Geh_JPEG/31112929740018466_1265233498.jpg"}, {"url": "http://shop1.phinf.naver.net/20240122_142/1705929358409wTYVQ_JPEG/5455576275694206_1220525852.jpg"}, {"url": "http://shop1.phinf.naver.net/20240122_112/1705929358680v11dv_JPEG/56541127863940465_1501463376.jpg"}, {"url": "http://shop1.phinf.naver.net/20240122_150/17059293589071DlnP_JPEG/56280742860303359_1054031388.jpg

In [105]:
upload_url = 'https://api.commerce.naver.com/external/v2/products'
headers = {
    'Authorization': f"Bearer {access_token}",
    'content-type': "application/json"
    }
response =requests.post(upload_url, json.dumps(post_body) ,headers=headers )

In [106]:
response.text

'{"code":"BAD_REQUEST","message":"잘못된 형식으로 JSON 데이터를 입력하셨습니다. - Cannot deserialize value of type `com.naver.ncp.sell.product.business.entity.product.enumeration.TaxType` from Array value (token `JsonToken.START_ARRAY`) [line: 1 column: 4,431]","timestamp":"2024-01-26T20:32:10.410+09:00"}'

개발 방향  : 아마존/라쿠텐 링크로 연결시킬수 있는 정보들은 생략하고 / 나머지 직접 써야하는 부분이나 이미지만 입력하면 바로 스마트 스토어에 올릴 수 있게 하는 프로그램으로 !


In [4]:
a.replace('\n','')

'"books": {    "returnCostReason": "string",    "noRefundReason": "string",    "qualityAssuranceStandard": "string",    "compensationProcedure": "string",    "troubleShootingContents": "string",    "title": "string",    "author": "string",    "publisher": "string",    "size": "string",    "pages": "string",    "components": "string",    "publishDate": "2024-01-13",    "publishDateText": "string",    "description": "string"},'

# 상품정보제공고시 정보

In [1]:
from ItemInfo_notification import Item_Info_Nofication

In [10]:
list(Item_Info_Nofication.keys())

['wear',
 'shoes',
 'bag',
 'fashionItems',
 'sleepingGear',
 'furniture',
 'imageAppliances',
 'homeAppliances',
 'seasonAppliances',
 'officeAppliances',
 'opticsAppliances',
 'microElectronics',
 'navigation',
 'carArticles',
 'medicalAppliances',
 'kitchenUtensils',
 'cosmetic',
 'jewellery',
 'food',
 'generalFood',
 'dietFood',
 'kids',
 'musicalInstrument',
 'sportsEquipment',
 'books',
 'rentalEtc',
 'digitalContents',
 'giftCard',
 'mobileCoupon',
 'movieShow',
 'etcService',
 'biochemistry',
 'biocidal',
 'cellPhone',
 'etc']

In [9]:
Item_Info_Nofication

{'wear': ['returnCostReason',
  'noRefundReason',
  'qualityAssuranceStandard',
  'compensationProcedure',
  'troubleShootingContents',
  'material',
  'color',
  'size',
  'manufacturer',
  'caution',
  'packDate',
  'packDateText',
  'warrantyPolicy',
  'afterServiceDirector'],
 'shoes': ['returnCostReason',
  'noRefundReason',
  'qualityAssuranceStandard',
  'compensationProcedure',
  'troubleShootingContents',
  'material',
  'color',
  'size',
  'height',
  'manufacturer',
  'caution',
  'warrantyPolicy',
  'afterServiceDirector'],
 'bag': ['returnCostReason',
  'noRefundReason',
  'qualityAssuranceStandard',
  'compensationProcedure',
  'troubleShootingContents',
  'type',
  'material',
  'color',
  'size',
  'manufacturer',
  'caution',
  'warrantyPolicy',
  'afterServiceDirector'],
 'fashionItems': ['returnCostReason',
  'noRefundReason',
  'qualityAssuranceStandard',
  'compensationProcedure',
  'troubleShootingContents',
  'type',
  'material',
  'size',
  'manufacturer',
  '